In [1]:
import sys
sys.path.append("..")


In [37]:
import pickle as pkl
import numpy as np
import pandas as pd

In [3]:
import meta_qa

In [4]:
from meta_qa.integrations import BigQueryIntegration

In [5]:
bqi = BigQueryIntegration("idwall-data", "dw_idwall")

In [6]:
#metadata = bqi.get_metadata()

In [7]:
#metadata.to_pickle("metadata.pkl")

In [8]:
metadata = pd.read_pickle("metadata.pkl")

In [9]:
tasks = bqi.get_tasks(metadata)

In [93]:
def lambda_test(x):
    
    location = x.name[0:3]
    column_name = x.name[-1]
    table_name = "`{}.{}.{}`".format(*location)
    
    column_params = {"table_name": table_name,
                     "column_name": column_name}
    
    operator = bqi.column_operators(column_params)
    
    return x.map(operator.function)


pipeline_out = tasks.head(5).apply(lambda_test, axis=1)

In [94]:
pipeline_out.iloc[0][0]

{'column_name': 'created',
 'location': '`idwall-data.dw_idwall.ft_attempts`',
 'operator': 'not_null',
 'raw_result': 1.0,
 'result': True}

In [95]:
pipeline_stack = pipeline_out.stack().droplevel(-1)
operators = pipeline_stack.apply(lambda x: x["operator"])

array(['not_null', 'lower_than', 'not_window_percentile_null',
       'larger_than'], dtype=object)

In [44]:
unique_ops = operators.values.flatten()
unique_ops = unique_ops[unique_ops != None]
unique_ops = np.unique(unique_ops)

In [170]:
output = []

for operator in operators.unique():
    a = (pd.DataFrame(pipeline_stack.where(operators == operator)
                                .dropna()).assign(operator=operator)
                                          .set_index("operator", append=True)
                                          .rename(columns={0: "result_object"})
                                          .unstack())

    a.columns = [operator]
    output.append(a)
pipeline_result = pd.concat(output, sort=False, axis=1)

In [177]:
pipeline_result.applymap(lambda x: "{} ({})".format(x["result"], x["raw_result"]) if type(x) is dict else "")

not_null  \
table_catalog table_schema table_name  column_name                
idwall-data   dw_idwall    ft_attempts created       True (1.0)   
                                       created_date  True (1.0)   
                                       created_hour  True (1.0)   
                                       end_date                   
                                       finished                   

                                                        lower_than  \
table_catalog table_schema table_name  column_name                   
idwall-data   dw_idwall    ft_attempts created            True (0)   
                                       created_date       True (0)   
                                       created_hour  False (77418)   
                                       end_date                      
                                       finished                      

                                                    not_window_percentile_null  \
table_catalog table_schema table_name  column_name                               
idwall-data   dw_idwall    ft_attempts created                                   
                                       created_date                              
                                       created_hour                              
                                       end_date                                  
                                       finished                     True (0.0)   

                                                    larger_than  
table_catalog table_schema table_name  column_name               
idwall-data   dw_idwall    ft_attempts created                   
                                       created_date              
                                       created_hour              
                                       end_date        True (0)  
                                       finished        True (0)